In [1]:
from keybert import KeyBERT
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import os
import sys
import json

In [2]:
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
grandparent_path = os.path.dirname(parent_path)
sys.path.append(grandparent_path)
sys.path.append(parent_path)

In [3]:
from users_manager.user_manager import UserManager_v1

username = "ChuckBOT"
user_manager = UserManager_v1(username=username)
session_id = None

session_df = user_manager.get_session_history_data_as_dataframe(session_id=session_id)

# Obtain only the message of role == User from session_df
user_messages = session_df[session_df["role"] == "User"]["message"].tolist()
user_messages

["Well, Eve, I'm feeling quite isolated and stressed out, to be honest. The long hours on the ship, the separation from my family, and the constant issues with the machinery department are really taking a toll on me. It's been hard to keep up with everything and I'm starting to feel the effects of it all.",
 "I've been trying to cope by keeping myself busy with work and focusing on the tasks at hand. But it's getting harder to keep up with that approach. I used to find solace in chatting with my crewmates, but lately, I've been feeling more isolated from them, too. I think I need to find some new ways to cope with this stress because it's starting to affect my performance at work as well.",
 "Yeah, I've been trying to schedule regular video calls with my family, and that does help to some extent. It's comforting to see their faces and hear their voices, even if it's through a screen. As for my crewmates, I think I might try organizing some group activities during our downtime, like mov

In [4]:
kw_model = KeyBERT('all-MiniLM-L6-v2')

In [5]:
path = os.path.join(grandparent_path, "documents", "Keywords", "TTE_Graph dataset_corrected.csv")
keywords_df = pd.read_csv(path)

keywords_df

Category  \
0                          Minimalist Lifestyle   
1   Minimizing & Evaluating Personal Belongings   
2                 Decluttering and Organization   
3               Minimalism & Intentional Living   
4                               Personal Growth   
5                                Mindful Living   
6                                      Emotions   
7                           Travel & Adventures   
8                          Financial Management   
9                                 Mental Health   
10                             Self-Improvement   
11                            Career and Financ   
12                  Mindfulness and Meditation    
13                          Health and Wellness   
14                                       Family   
15                   Happiness and Fulfillment    
16                          Hobbies & Interests   
17                                         Work   
18                       Learning and Education   
19                               Spiritual Life   
20                            Stress Management   
21                              Daily Routines    
22                            Coping Strategies   
23                        Personal Information    
24                      Recreational Activities   
25                         Exercise and Fitness   
26                         Budgeting and Saving   
27                         Eco-Friendly Living    
28                          Innovative Thinking   
29                               Relationships    
30                             Physical Health    
31                        Personal Development    
32                              Personal Growth   
33                            Anxiety and Fears   
34                                Achievements    
35                              Medical History   
36                          Goals & Aspirations   
37                            Health & Wellness   
38                Home & Living Life Management   
39                                  Social Life   
40                       Recreation and Leisure   
41                          Resilience and Grit   
42                            Work-Life Balance   
43                               Dreams (Sleep)   
44                           Career Progression   
45                               Mental Health    
46                                     Finance    
47                            Creative Projects   
48                    Innovation and Creativity   
49                            Lifelong Learning   
50                             Mental Wellness    
51                       Philosophical Beliefs    
52                            Digital Footprint   
53                                  Personality   
54                            Skills & Talents    
55                             Personal Finance   
56                                    Education   
57                       Challenges & Setbacks    
58                         Community Engagement   

                                          Parent Node  \
0                                Personal Development   
1                                Personal Development   
2                                Personal Development   
3                      Personal Development/Lifestyle   
4                                Personal Development   
5                                Personal Development   
6                                       Mental Health   
7                                     Personal Growth   
8                                    Personal Finance   
9                                 Health and Wellness   
10                               Personal Development   
11                               Personal Development   
12                               Personal Development   
13                                          Self-Care   
14                             Personal Relationships   
15                               Personal Development   
16       

In [6]:
nodes = keywords_df["Category"].unique().tolist()
len(nodes)
description = keywords_df["Description"].tolist()
parent_nodes = keywords_df["Parent Node"].tolist()

In [7]:
def get_nodes_from_description(messages_list, description_list, threshold=0.5):
    description_embeddings, _ = kw_model.extract_embeddings(description_list)
    messages_embeddings, _ = kw_model.extract_embeddings(messages_list)
    
    # Now here we need to compute the cosine similarity between the nodes and the messages
    similarity_matrix = cosine_similarity(messages_embeddings, description_embeddings)
    
    # Now we need to normalize the similarity matrix for every row
    normalized_similarity_matrix =  (similarity_matrix - similarity_matrix.min(axis=1,keepdims=True)) / (similarity_matrix.max(axis=1,keepdims=True) - similarity_matrix.min(axis=1,keepdims=True))

    # Now we need to obtain the index of the nodes where the similarity is over a certain threshold)
    idx = np.where(normalized_similarity_matrix > threshold)
    
    # Return idx, and the normalized_similarity_matrix
    return idx , normalized_similarity_matrix

In [13]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_nodes_from_category(messages_list, node_list, threshold=None, normalize=True, n_max=None):
    nodes_embeddings, _ = kw_model.extract_embeddings(node_list)
    messages_embeddings, _ = kw_model.extract_embeddings(messages_list)
    
    # Calcular la similitud coseno entre los nodos y los mensajes
    similarity_matrix = cosine_similarity(messages_embeddings, nodes_embeddings)
    
    if normalize:
        # Normalizar la matriz de similitud para cada fila
        similarity_matrix = (similarity_matrix - similarity_matrix.min(axis=1, keepdims=True)) / \
                            (similarity_matrix.max(axis=1, keepdims=True) - similarity_matrix.min(axis=1, keepdims=True))

    if threshold is None:
        threshold = 1 / len(node_list)
        
    similarity_matrix[similarity_matrix < threshold] = 0
    
    
    
    # Utilizar argsort para obtener los índices de los n_max elementos más grandes en cada fila
    if n_max is None:
        n_max = len(node_list)
        
    idx = np.where(similarity_matrix >= threshold)
    
    # Crear listas para almacenar los resultados
    message_ids = []
    nodes_list = []
    
    for i, row in enumerate(idx):
        # Obtener las listas de nodos y similitudes
        nodes = [node_list[j] for j in row]
        
        # Almacenar en las listas
        message_ids.extend([i] * len(nodes))
        nodes_list.extend(nodes)
    
    # Crear un DataFrame
    result_df = pd.DataFrame({'message_id': message_ids, 'node': nodes_list})
    
    # Devolver el diccionario y la matriz de similitud normalizada
    return result_df, similarity_matrix


In [20]:
result_dict, similarity_matrix = get_nodes_from_category(user_messages, nodes, threshold=0.45, normalize=False, n_max=3)
result_dict

message_id                                         node
0            0  Minimizing & Evaluating Personal Belongings
1            0  Minimizing & Evaluating Personal Belongings
2            0                          Travel & Adventures
3            0                          Travel & Adventures
4            0                                       Family
5            0                                       Family
6            0                            Stress Management
7            1                            Stress Management
8            1                            Coping Strategies
9            1                              Daily Routines 
10           1                         Exercise and Fitness
11           1                            Stress Management
12           1                            Coping Strategies
13           1                            Stress Management

In [32]:
result_dict, similarity_matrix = get_nodes_from_category(user_messages, nodes, threshold=0.1, normalize=False, n_max=None)

# Now we to show the message and the nodes that are related to it and the parent node 

keys = list(result_dict.keys())
for i in range(len(user_messages)):
    message = user_messages[i]
    key = keys[i]
    print(f"Message: {message}")
    print(f"Related nodes: {result_dict[key]['nodes']}")
    print(f"Nodes similarities: {result_dict[key]['similarities']}")
    parent_nodes = keywords_df[keywords_df["Category"].isin(result_dict[key]['nodes'])]["Parent Node"].tolist()
    print(f"Parent Nodes:{parent_nodes}")

Message: Well, Eve, I'm feeling quite isolated and stressed out, to be honest. The long hours on the ship, the separation from my family, and the constant issues with the machinery department are really taking a toll on me. It's been hard to keep up with everything and I'm starting to feel the effects of it all.
Related nodes: ['Innovative Thinking', 'Exercise and Fitness', 'Learning and Education', 'Budgeting and Saving', 'Hobbies & Interests', 'Achievements ', 'Medical History', 'Goals & Aspirations', 'Dreams (Sleep)', 'Career and Financ', 'Finance ', 'Creative Projects', 'Financial Management', 'Innovation and Creativity', 'Lifelong Learning', 'Philosophical Beliefs ', 'Digital Footprint', 'Skills & Talents ', 'Personal Finance', 'Education', 'Recreational Activities', 'Personal Information ', 'Community Engagement', 'Minimalism & Intentional Living', 'Recreation and Leisure', 'Personality', 'Personal Growth', 'Physical Health ', 'Personal Development ', 'Minimizing & Evaluating Per

In [38]:
# Here we need to create a dataframe for every node and parent node and the number_message, but 



In [37]:
keywords

nodes  \
0   [Innovative Thinking, Exercise and Fitness, Le...   
1   [Innovative Thinking, Goals & Aspirations, Phy...   
2   [Innovative Thinking, Dreams (Sleep), Personal...   
3   [Dreams (Sleep), Philosophical Beliefs , Achie...   
4   [Innovative Thinking, Budgeting and Saving, Re...   
5   [Innovative Thinking, Goals & Aspirations, Hea...   
6   [Innovative Thinking, Challenges & Setbacks , ...   
7   [Innovative Thinking, Relationships , Anxiety ...   
8   [Minimalist Lifestyle, Personal Development , ...   
9   [Innovative Thinking, Physical Health , Achiev...   
10  [Innovative Thinking, Relationships , Learning...   
11  [Innovative Thinking, Budgeting and Saving, Ec...   
12  [Minimalist Lifestyle, Challenges & Setbacks ,...   
13  [Minimalist Lifestyle, Challenges & Setbacks ,...   
14  [Innovative Thinking, Budgeting and Saving, Re...   
15  [Innovation and Creativity, Goals & Aspiration...   
16  [Innovative Thinking, Relationships , Personal...   
17  [Innovative Thinking, Medical History, Goals &...   
18  [Innovative Thinking, Budgeting and Saving, Ex...   
19  [Work, Goals & Aspirations, Physical Health , ...   
20  [Innovative Thinking, Relationships , Achievem...   
21  [Minimalist Lifestyle, Relationships , Persona...   
22  [Minimalist Lifestyle, Eco-Friendly Living , P...   

                                         similarities  
0   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10672214...  
4   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
5   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
6   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
7   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
8   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
9   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
10  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
11  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
12  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
13  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
14  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
15  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
16  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
17  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
18  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
19  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
20  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
21  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
22  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...